# Install or upgrade libraries

It might be that you are running with the latest libraries and that they all work together fine.

Running the following cell takes a minute or so but ensures that you have a consistent set of python tools.

In [ ]:
import sys
print(f"{sys.version = }\n")


In [ ]:
# If there are issues with fsspect-xrootd not being found, run this outside of Jupyter-notebook and restart
# !pip install --upgrade fsspec-xrootd

In [ ]:
#'''
!pip install --upgrade pip

!pip install futures

!pip install --user --upgrade coffea

!pip install --upgrade awkward
!pip install --upgrade uproot

!pip install --upgrade fsspec-xrootd

!pip install vector

!pip install --upgrade pandas


!pip install --upgrade matplotlib
#'''

We've also prepared some helper code that makes it easier to work with the data in this lesson.

You can see the code [here](https://github.com/cms-opendata-workshop/workshop2024-lesson-event-selection/blob/main/instructors/dpoa_workshop_utilities.py) but we will explain the functions and data objects in this notebook.

Let's download it first.

In [ ]:
#!wget https://raw.githubusercontent.com/cms-opendata-workshop/workshop2024-lesson-event-selection/main/instructors/dpoa_workshop_utilities.py

## Imports

Import all the libraries we will need and check their versions, in case you run into issues.

In [ ]:
%load_ext autoreload
%autoreload 2

# The classics
import numpy as np
import matplotlib.pylab as plt
import matplotlib # To get the version

import pandas as pd

# The newcomers
import awkward as ak
import uproot

import vector
vector.register_awkward()

import requests
import os

import time

import json

import dpoa_workshop_utilities
from dpoa_workshop_utilities import nanoaod_filenames
from dpoa_workshop_utilities import get_files_for_dataset
from dpoa_workshop_utilities import pretty_print
from dpoa_workshop_utilities import build_lumi_mask

import sys

In [ ]:
print("Versions --------\n")
print(f"{sys.version = }\n")
print(f"{ak.__version__ = }\n")
print(f"{uproot.__version__ = }\n")
print(f"{np.__version__ = }\n")
print(f"{matplotlib.__version__ = }\n")
print(f"{vector.__version__ = }\n")
print(f"{pd.__version__ = }\n")

# Opening a file

Let's open and explore a sample file.

We'll be getting the data from [here](https://opendata.cern.ch/record/67993).

This is some Monte Carlo that contains simulations of a top-antitop pair being created in a proton-proton collision at CMS.

One top decays leptonically and the other decays hadronically.

**Do you know what leptonically and hadronically mean? If not, do a bit of research.**

When you go to open the file, it might take 10-30 seconds at this step if you are working with the larger file.

In [ ]:
# For testing
# Big file
#filename = 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/08FCB2ED-176B-064B-85AB-37B898773B98.root'

# Smaller file, better for prototyping your code as things will run faster
filename = 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/7D120E49-E712-B74B-9E1C-67F2D0057995.root'

# print(f"Opening...{filename}")
# f = uproot.open(filename)

# events = f['Events']

# nevents = events.num_entries

# print(f"{nevents = }")

The `events` object is a `TTree` implementation in python and behaves like a dictionary. This means
we can get all the keys if we want.

In [ ]:
# Uncomment the following line to print all the keys

#print(events.keys())

Again, we have provided you with a helper function called `pretty_print` that will print subsets of the keys, based on strings
that you require or ignore.

It will also format that output based on how many characters you want in a column (you are limited to 80 characters per line).

Here is some example usage.

In [ ]:
# Pretty print all the keys with the default format
#pretty_print(events.keys())

# Pretty print keys with 30 characters per column, for keys that contain `FatJet`
#pretty_print(events.keys(), fmt='30s', require='FatJet')

# Pretty print keys with 40 characters per column, for keys that contain `Muon` and `Iso` but ignore ones with `HLT`
#pretty_print(events.keys(), fmt='40s', require=['Muon'], ignore='HLT')

# Pretty print keys with 40 characters per column, for keys that contain `HLT` and `TkMu50`
#pretty_print(events.keys(), fmt='40s', require=['HLT', 'TkMu50'])

# Pretty print keys with 40 characters per column, for keys that contain `HLT`
#pretty_print(events.keys(), fmt='40s', require='HLT')

# Pretty print keys with 40 characters per column, for keys that contain `Jet_` but ignore ones with `Fat`
#pretty_print(events.keys(), fmt='40s', require='Jet_', ignore='Fat')

# Pretty print keys with 40 characters per column, for keys that contain `PuppiMET` but ignore ones with `Raw`
#pretty_print(events.keys(), fmt='40s', require='PuppiMET', ignore='Raw')

## Extract some data

We're going to pull out subsets of the data in order to do our analysis.

As a reminder, you can find a list of the variable names in each dataset on the CERN Open Data Portal page for that dataset, for example, [here](https://opendata.cern.ch/eos/opendata/cms/dataset-semantics/NanoAODSIM/75156/ZprimeToTT_M2000_W20_TuneCP2_PSweights_13TeV-madgraph-pythiaMLM-pythia8_doc.html).

We're going to work with the following sets of variables
* `FatJet` for jets that are merges
* `Jet` for non-merged jets
* `Muon` for muons
* `PuppiMET` which is missing energy in the transverse plane (MET) for pileup per particle identification (Puppi)

Running this cell might take a little bit if you are running over the bigger file. However, once you pull out the values, later calculations are much faster.

In [ ]:
# # Jets ---------------------------------------------------
# # B-tagging variable
# jet_btag = events['Jet_btagDeepB'].array()

# # Measure of quality of measurement of jet
# jet_jetid = events['Jet_jetId'].array()

# # 4-momentum in pt, eta, phi, mass
# jet_pt = events['Jet_pt'].array()
# jet_eta = events['Jet_eta'].array()
# jet_phi = events['Jet_phi'].array()
# jet_mass = events['Jet_mass'].array()


# # Muons ---------------------------------------------------
# # Muon isolation
# muon_iso = events['Muon_miniIsoId'].array()

# # Measure of quality of how well the muon is reconstructed
# muon_tightId = events['Muon_tightId'].array()

# # 4-momentum in pt, eta, phi, mass
# muon_pt = events['Muon_pt'].array()
# muon_eta = events['Muon_eta'].array()
# muon_phi = events['Muon_phi'].array()
# muon_mass = events['Muon_mass'].array()


# # MET ------------------------------------------------------
# # 3-momentum in pt, eta, phi, mass
# met_pt = events['PuppiMET_pt'].array()
# met_eta = 0*events['PuppiMET_pt'].array()  # Fix this to be 0
# met_phi = events['PuppiMET_phi'].array()

# What comes next?

In [ ]:
# Smaller file, better for prototyping your code as things will run faster
small_file = 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/7D120E49-E712-B74B-9E1C-67F2D0057995.root'
# Big file
big_file = 'root://eospublic.cern.ch//eos/opendata/cms/mc/RunIISummer20UL16NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v1/120000/08FCB2ED-176B-064B-85AB-37B898773B98.root'
# Real Data
real_file = 'root://eospublic.cern.ch//eos/opendata/cms/Run2016H/JetHT/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/130000/0290F73B-A51C-A441-AEC1-8429F9CC8AA8.root'

## Functions

In [ ]:
def process_file(filename):
    """
    Root file processing function;
    
    ############################
    ########## INPUTS ##########
    ############################
    
    filename (str, default=None) - Full root file destination

    #############################
    ########## RETURNS ##########
    #############################
    
    events (uproot.model (Tree)) - Root file keys
    """
    ############################################
    ########## OPENING SPECIFIED FILE ##########
    ############################################
    print(f"Opening...{filename}")
    
    try:
        f = uproot.open(filename)
    except:
        print(f"Could not open {filename}")
        return None

    ####################################################################
    ########## ACCESSING EVENTS AND MAKING SPECIFIC VARIABLES ##########_jet_cut
    ####################################################################
    
    events = f['Events']

    nevents = events.num_entries

    print(f"{nevents = }")


    
    return events

In [ ]:
def plot_func(events=None, pt_cut_vals=[0], btag_cut="Tight", reconstruct=False, top_had_cand=None, w_had_cand=None, top_lep_cand=None, w_lep_cand=None):
    """
    Plotting Function

    Plots histograms of Jet and Muon Transverse Momentum and respective numbers
    If 'reconstruct
    
    ############################
    ########## INPUTS ##########
    ############################
    
    events (uproot.model (Tree), default=None) - Root file keys; Can be entered to skip re-processing of root file
    
    pt_cut_vals (int/float list, default=[0]) - Transverse momentum cut values
    
    btag_cut (str, default="Tight") - None, Loose, Medium, Tight cut using Jet_btagDeepB and Jet_btagDeepFlavB

    reconstruct (bool, default=False) - Turns on invariant mass plots, otherwise jets and muons plot

    ## IGNORE THE FOLLOWING INPUTS ##
    top_had_cand,
    w_had_cand,
    top_lep_cand,
    w_lep_cand

    #############################
    ########## RETURNS ##########
    #############################

    None.
    """
    print()
    print("Plotting in progress...")
    print()
    
    btag_cut_dict = {
        "None"  : [0.0000, 0.0000, "black"],
        "none"  : [0.0000, 0.0000, "black"],
        
        "Loose" : [0.1918, 0.0480, "purple"],
        "loose" : [0.1918, 0.0480, "purple"],
        
        "Medium": [0.5847, 0.2489, "yellow"],
        "medium": [0.5847, 0.2489, "yellow"],
        
        "Tight" : [0.8767, 0.6377, "red"],
        "tight" : [0.8767, 0.6377, "red"]
    }
    btag_val = btag_cut_dict[btag_cut][0] # Options above
    
    accepted_topq = 172.76 # GeV
    accepted_wbos = 80.4 # GeV
    
    if reconstruct == True:
        print("Plotting Invariant Mass...")
        print()
        plt.figure(figsize=(20,5))
        plt.suptitle("Hadronic Reconstruction",fontsize=20)
        plt.subplot(1,2,1)
        plt.title("Top Quark Candidate",fontsize=16)
        counts1, bins1, patches1 = plt.hist(top_had_cand.mass, bins=100, range=(0,500),label=f"btag cut: {btag_val}")
        
        
        peak_bin_idx1 = np.argmax(counts1) ; peak_pos1 = (bins1[peak_bin_idx1] + bins1[peak_bin_idx1 + 1]) / 2
        
        plt.axvline(peak_pos1,color="salmon",linestyle="--",label=f"Peak: {peak_pos1:.2f} GeV")
        plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value: {accepted_topq} GeV")
        
        
        plt.xticks(np.arange(125,200,25),minor=True)
        plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
        plt.ylabel("Events",fontsize=14)
        plt.legend()
        ##########################################################
        plt.subplot(1,2,2)
        plt.title("W Boson Candidate",fontsize=16)
        counts2, bins2, patches2 = plt.hist(w_had_cand.mass, bins=100, range=(75,275),label=f"btag cut: {btag_val}")
        
        
        peak_bin_idx2 = np.argmax(counts2) ; peak_pos2 = (bins2[peak_bin_idx2] + bins2[peak_bin_idx2 + 1]) / 2
        
        plt.axvline(peak_pos2,color="salmon",linestyle="--",label=f"Peak: {peak_pos2:.2f} GeV")
        plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value: {accepted_wbos} GeV")
        
        
        #plt.xticks(np.arange(125,200,25),minor=True)
        plt.yscale('log')
        plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
        plt.ylabel("Events",fontsize=14)
        plt.legend()
        ###############################################################################################################
        ###############################################################################################################
        ###############################################################################################################
        ###############################################################################################################
        plt.figure(figsize=(20,5))
        plt.suptitle("Leptonic Reconstruction",fontsize=20)
        plt.subplot(1,2,1)
        plt.title("Top Quark Candidate",fontsize=16)
        counts1, bins1, patches1 = plt.hist(top_lep_cand.mass, bins=100, range=(0,500),label=f"btag cut: {btag_val}")
        
        
        peak_bin_idx1 = np.argmax(counts1) ; peak_pos1 = (bins1[peak_bin_idx1] + bins1[peak_bin_idx1 + 1]) / 2
        
        plt.axvline(peak_pos1,color="salmon",linestyle="--",label=f"Peak: {peak_pos1:.2f} GeV")
        plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value: {accepted_topq} GeV")
        
        
        plt.xticks(np.arange(125,200,25),minor=True)
        plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
        plt.ylabel("Events",fontsize=14)
        plt.legend();
        ##########################################################
        plt.subplot(1,2,2)
        plt.title("W Boson Candidate",fontsize=16)
        counts2, bins2, patches2 = plt.hist(w_lep_cand.mass, bins=100, range=(75,275),label=f"btag cut: {btag_val}")
        
        
        peak_bin_idx2 = np.argmax(counts2) ; peak_pos2 = (bins2[peak_bin_idx2] + bins2[peak_bin_idx2 + 1]) / 2
        
        plt.axvline(peak_pos2,color="salmon",linestyle="--",label=f"Peak: {peak_pos2:.2f} GeV")
        plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value: {accepted_wbos} GeV")
        
        
        #plt.xticks(np.arange(125,200,25),minor=True)
        plt.yscale('log')
        plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
        plt.ylabel("Events",fontsize=14)
        plt.legend();

    ################################################
    ########## VARIABLES FROM EVENTS FILE ##########
    ################################################
    else:
        # Muons -------------------------------------------------------
        muon_pt = events['Muon_pt'].array()
        # muon_eta = events['Muon_eta'].array()
        # muon_phi = events['Muon_phi'].array()
        # muon_mass = events['Muon_mass'].array()
    
        # muon_iso = events['Muon_miniIsoId'].array()
    
        # muon_tightId = events['Muon_tightId'].array()
    
        
        # Jets -------------------------------------------------------
        jet_btag = events['Jet_btagDeepB'].array()
        jet_btagflav = events['Jet_btagDeepFlavB'].array()
        # jet_jetid = events['Jet_jetId'].array()
    
        jet_pt = events['Jet_pt'].array()
        # jet_eta = events['Jet_eta'].array()
        # jet_phi = events['Jet_phi'].array()
        # jet_mass = events['Jet_mass'].array()
    
        ###################################
        ########## APPLYING CUTS ##########
        ###################################
    
        for cut in pt_cut_vals:
            pt_jet_cut = jet_pt > cut
            
            ## B-Tagging ----------------------------------------------------
            
            # if btag_cut == "tight" or btag_cut == "Tight":
            #     deep_b_tag = events["Jet_btagDeepB"].array() > 0.8767
            #     deep_flavb_tag = events["Jet_btagDeepFlavB"].array() > 0.6377
            #     btag = deep_b_tag & deep_flavb_tag
    
            #     cut_jets = jet_pt[pt_jet_cut[btag]]
    
            # elif btag_cut == "medium" or btag_cut == "Medium":
            #     deep_b_tag = events["Jet_btagDeepB"].array() > 0.5847
            #     deep_flavb_tag = events["Jet_btagDeepFlavB"].array() > 0.2489
            #     btag = deep_b_tag & deep_flavb_tag
    
            #     cut_jets = jet_pt[pt_jet_cut[btag]]
    
            # elif btag_cut == "loose" or btag_cut == "Loose":
            #     deep_b_tag = events["Jet_btagDeepB"].array() > 0.1918
            #     deep_flavb_tag = events["Jet_btagDeepFlavB"].array() > 0.0480
            #     btag = deep_b_tag & deep_flavb_tag
    
            #     cut_jets = jet_pt[pt_jet_cut[btag]]
    
            # else:
            #     cut_jets = jet_pt[pt_jet_cut]
    
            
            pt_muon_cut = muon_pt > cut
            cut_muons = muon_pt[pt_muon_cut]

            btag = (jet_btag > btag_cut_dict[btag_cut][0]) & (jet_btagflav > btag_cut_dict[btag_cut][1])
            cut_jets = jet_pt[pt_jet_cut[btag]]
            ##############################
            ########## PLOTTING ##########
            ##############################
            
            fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, figsize=(20,5), tight_layout=True)
            
            fig.text(0.27,0.92, f"Jets : $p_T$>{cut} (GeV/$c$) | BTag: {btag_cut}", ha='center', fontsize=18)
            fig.text(0.77, 0.92, f"Muons : $p_T$>{cut} (GeV/$c$)", ha='center', fontsize=18)
            
            ## Jets -------------------------------------------------------------
            ax1.hist(ak.flatten(cut_jets), 
                     bins=100,label=f"Number of Jets:{ak.sum(ak.num(cut_jets))}",
                    range=(0,400))
            ax1.set_xlabel("Transverse Momentum",fontsize=14)
            ax1.set_ylabel("Counts",fontsize=14)
            ax1.legend()
    
            ax2.hist(ak.num(cut_jets), 
                     bins=100,label=f"Number of Jets:{ak.sum(ak.num(cut_jets))}",
                    range=(0,10))
            ax2.set_xlabel("Number of Jets",fontsize=14)
            #ax2.set_ylabel("Counts")
            #ax2.title(f"Jets : $p_T$>{cut} (GeV/$c$)")
            ax2.legend()
    
            ## Muons --------------------------------------------------------------
            ax3.hist(ak.flatten(cut_muons), 
                     bins=100,label=f"Number of Muons:{ak.sum(ak.num(cut_muons))}",
                    range=(0,400), color="darksalmon")
            ax3.set_xlabel("Transverse Momentum",fontsize=14)
            #ax3.set_ylabel("Counts")
            ax3.legend()
    
            ax4.hist(ak.num(cut_muons), 
                     bins=100,label=f"Number of Muons:{ak.sum(ak.num(cut_muons))}",
                    range=(0,5), color="darksalmon")
            ax4.set_xlabel("Number of Muons",fontsize=14)
            #ax4.set_ylabel("Counts")
            #ax4.title(f"Muons : $p_T$>{cut} (GeV/$c$)")
            ax4.legend()
    
            plt.tight_layout(rect=[0,0,1,0.93])


In [ ]:
def events_to_list(events):
    """
    Places required events into in awkward array 4momenta lists via ak.zip that are necessary for reconstruction

    ############################
    ########## INPUTS ##########
    ############################

    events - TTREE object of particle collisions

    #############################
    ########## RETURNS ##########
    #############################

    muons - muon 4 momenta array
    jets - jet 4 momenta array
    met - missing transverse energy 4 momenta array (assumed to be a neutrino)
    """
    # Muons -------------------------------------------------------
    print("Accessing events from tree...")
    access_time = time.time()
    muon_pt = events['Muon_pt'].array()
    muon_eta = events['Muon_eta'].array()
    muon_phi = events['Muon_phi'].array()
    muon_mass = events['Muon_mass'].array()
    
    print(f"It took {time.time() - access_time:.2f} seconds to access Muons.")
        
    # Jets -------------------------------------------------------    
    jet_pt = events['Jet_pt'].array()
    jet_eta = events['Jet_eta'].array()
    jet_phi = events['Jet_phi'].array()
    jet_mass = events['Jet_mass'].array()
    jets_btagDeepB = events["Jet_btagDeepB"].array()
    jets_btagDeepFlavB = events["Jet_btagDeepFlavB"].array()
    
    print(f"It took {time.time() - access_time:.2f} seconds to access Jets.")
    
    # MET ---------------------------------------------------------
    met_pt = events['PuppiMET_pt'].array()
    met_eta = 0*events['PuppiMET_pt'].array()  # Fix this to be 0
    met_phi = events['PuppiMET_phi'].array() 
    
    print(f"It took {time.time() - access_time:.2f} seconds to access MET.")
    print()
    
    ######################################
    print()
    print("Zipping data...")
    muons = ak.zip(
        {"pt": muon_pt, 
         "eta": muon_eta, 
         "phi": muon_phi, 
         "mass": muon_mass},
        with_name="Momentum4D",
    )
    
    jets = ak.zip(
        {"pt": jet_pt, 
         "eta": jet_eta, 
         "phi": jet_phi, 
         "mass": jet_mass,
         "btagDeepB": jets_btagDeepB,
         "btagDeepFlavB": jets_btagDeepFlavB},
        with_name="Momentum4D",
    )
    
    met = ak.zip(
        {"pt": met_pt, 
         "eta": met_eta, 
         "phi": met_phi, 
         "mass": np.zeros(len(met_phi))}, # We assume this is a neutrino with 0 mass
        with_name="Momentum4D",
    )
    print(f"Data zipped in {time.time() - access_time:.2f} seconds.")

    return muons, jets, met

In [ ]:
def reconstructor(muons, jets, met, btag_cut):
    """
    Uses Muon, Jet, and MET data from the events tree to reconstruct the Top Quark and W Boson hadronically and leptonically.

    ############################
    ########## INPUTS ##########
    ############################

    muons - muon 4 momenta array
    jets - jet 4 momenta array
    met - missing transverse energy 4 momenta array (assumed to be a neutrino)

    #############################
    ########## RETURNS ##########
    #############################

    top_had_cand (Awkward Array Record Type) - values of top quark from hadronic reconstruction
    w_had_cand (Awkward Array Record Type) - values of w boson from hadronic reconstruction
    
    top_lep_cand (Awkward Array Record Type) - values of top quark from leptonic reconstruction
    w_lep_cand (Awkward Array Record Type) - values of w boson from leptonic reconstruction
    """
    had_time = time.time()
                #############################################
                ########## HADRONIC RECONSTRUCTION ##########
                #############################################
    print("=============================================================")
    print("Starting Hadronic Reconstruction.")

    
    #################
    ##### CUTS ######
    #################
    print()
    print("Beginning cuts and selections...")
    
    muon_mask =          (muons.pt > 20) & (np.abs(muons.eta) < 2.1)
    jet_kinematic_mask = (jets.pt > 20) & (np.abs(jets.eta) < 2.4)
    
    cut_muons = muons[muon_mask]
    cut_jets =  jets[jet_kinematic_mask]

    btag_cut_dict = {
        "None"  : [0.0000, 0.0000, "black"],
        "none"  : [0.0000, 0.0000, "black"],
        
        "Loose" : [0.1918, 0.0480, "purple"],
        "loose" : [0.1918, 0.0480, "purple"],
        
        "Medium": [0.5847, 0.2489, "yellow"],
        "medium": [0.5847, 0.2489, "yellow"],
        
        "Tight" : [0.8767, 0.6377, "red"],
        "tight" : [0.8767, 0.6377, "red"]
    }
    btag_val = btag_cut_dict[btag_cut][0] # Options above
    
    n_muons = ak.num(cut_muons)
    n_jets =  ak.num(cut_jets)
    
    event_mask = (n_muons == 1) & (n_jets >= 4) & (met.pt > 30) # MET cut done separately here for sizing reasons
    
    selected_jets =  cut_jets[event_mask]
    selected_muons = cut_muons[event_mask]
    selected_met =   met[event_mask]
    
    jet_bjet_mask =     (selected_jets.btagDeepB > btag_val)
    jet_not_bjet_mask = (selected_jets.btagDeepB < 0.5)
    
    jets_b =     selected_jets[jet_bjet_mask]
    jets_not_b = selected_jets[jet_not_bjet_mask]
    
    n_bjets =     ak.num(jets_b)
    n_not_bjets = ak.num(jets_not_b)
    
    mask_jets = (n_bjets >= 2) & (n_not_bjets >=2)

    print(f"Finished cuts and selections after {time.time()-had_time:.2f} seconds.")
    print()
    ##########################
    ###### COMBINATIONS ######
    ##########################
    print("Beginning combinations of bjets and non-bjets...")
    
    bjets_combos =     ak.combinations(jets_b[mask_jets], 2, fields=["b1","b2"])
    not_bjets_combos = ak.combinations(jets_not_b[mask_jets], 2, fields=["j1","j2"])
    combo = ak.cartesian({"bjets":bjets_combos, "jets":not_bjets_combos})
    
    print(f"Finished combinations after {time.time()-had_time:.2f} seconds.")
    print()

    ###################
    ##### W BOSON #####
    ###################
    print()
    print("Reconstructing W Boson...")
    
    w_had_cand = ak.flatten(combo.jets.j1 + combo.jets.j2)
    print(f"Num candidates before cut: {len(w_had_cand.mass)}")
    print(f"Avg mass before cut: {np.mean(w_had_cand.mass):.2f}")
    
    print()
    
    w_mass_mask = (w_had_cand.mass > 60) & (w_had_cand.mass < 100)
    print(f"Num candidates after cut: {len(w_had_cand[w_mass_mask].mass)}")
    print(f"Avg mass after cut: {np.mean(w_had_cand[w_mass_mask].mass):.2f}")
    print()

    #####################
    ##### TOP QUARK #####
    #####################
    print()
    print("Reconstructing Top Quark...")
    
    b1_combo =  ak.flatten(combo.bjets.b1)
    b2_combo =  ak.flatten(combo.bjets.b2)
    top_had_1 = b1_combo[w_mass_mask] + w_had_cand[w_mass_mask]
    top_had_2 = b2_combo[w_mass_mask] + w_had_cand[w_mass_mask]

    # FINDING PEAK MASS VALUES
    counts1, bin_edges1 = np.histogram(top_had_1.mass, bins=100, range=(0,500))
    peak_ind1 = np.argmax(counts1) ; peak_pos1 = (bin_edges1[peak_ind1] + bin_edges1[peak_ind1 + 1])/2

    counts2, bin_edges2 = np.histogram(top_had_2.mass, bins=100, range=(0,500))
    peak_ind2 = np.argmax(counts2) ; peak_pos2 = (bin_edges2[peak_ind2] + bin_edges2[peak_ind2 + 1])/2

    accepted_topq = 172.76 # GeV
    accepted_wbos = 80.4 # GeV

    if abs(accepted_topq - peak_pos1) < abs(accepted_topq - peak_pos2):
        top_had_cand = top_had_1
        lep_bjet_set = combo.bjets.b2
        print("Top Quark Candidate 1 is closer to accepted value.")
        print("Bjet 2 has been chosen for leptonic reconstruction.")
    else:
        top_had_cand = top_had_2
        lep_bjet_set = combo.bjets.b1
        print("Top Quark Candidate 2 is closer to accepted value.")
        print("Bjet 1 has been chosen for leptonic reconstruction.")
    
    print()
    print(f"Finished Hadronic Reconstruction after {time.time()-had_time:.2f} seconds.")
    print()
    
                #############################################
                ########## LEPTONIC RECONSTRUCTION ##########
                #############################################
    lep_time = time.time()
    print()
    print("Starting Leptonic Reconstruction...")
    #################
    ##### MUONS #####
    #################
    lep_mu = ak.zip(
        {"px": selected_muons[mask_jets][:,0].px,
         "py": selected_muons[mask_jets][:,0].py,
         "pz": selected_muons[mask_jets][:,0].pz,
         "E":  selected_muons[mask_jets][:,0].energy},
        with_name="Momentum4D"
    )
    
    #####################
    ##### NEUTRINOS #####
    #####################
    lep_nu = ak.zip(
        {"px": selected_met[mask_jets].px,
         "py": selected_met[mask_jets].py,
         "pz": np.zeros(len(selected_met[mask_jets].py)),
         "E":  np.zeros(len(selected_met[mask_jets].py))},
        with_name="Momentum4D"
    )
    
    #################
    ##### BJETS #####
    #################
    lep_bjet = ak.zip(
        {"px": lep_bjet_set[:,0].px,
         "py": lep_bjet_set[:,0].py,
         "pz": lep_bjet_set[:,0].pz,
         "E":  lep_bjet_set[:,0].energy},
        with_name="Momentum4D"
    )

    #################################################
    ##### CALCULATION OF NEUTRINO PZ AND ENERGY #####
    #################################################
    print("Calculating neutrino 4momenta...")
    
    MW = 80.4 # GeV
    mu_dot_nu = lep_mu.px*lep_nu.px + lep_mu.py*lep_nu.py # dot product between mu <px,py> and nu <px,py>

    
    #########################################
    ##### QUADRATIC EQUATION VARIABLES ######
    #########################################
    Dtmp = MW**2 - lep_mu.mass**2 + 2*(mu_dot_nu)
    Atmp = 4*(lep_mu.energy**2 - lep_mu.pz**2)
    Btmp = -4 * Dtmp * lep_mu.pz
    Ctmp = 4 * lep_mu.energy**2 * lep_nu.pt**2 - Dtmp**2
    
    disc = Btmp**2 - 4*Atmp*Ctmp
    
    nu_pz1 = (-Btmp + np.sqrt(disc))/(2*Atmp)
    nu_pz2 = (-Btmp - np.sqrt(disc))/(2*Atmp)
    
    ########################################
    ##### GRABBING ONLY REAL SOLUTIONS #####
    ########################################
    real_sol = disc >= 0
    
    # CONDITIONAL TO TAKE THE BEST SOLUTION FOR NU PZ
    lep_nu["pz"] = ak.where(real_sol, # equal to where real solution is true
                            ak.where(abs(nu_pz1) < abs(nu_pz2), nu_pz1, nu_pz2), # deciding which one is greater to choose the best solution
                            -Btmp/(2*Atmp) # Otherwise forcing real solution
    )
    
    # CALCULATIING THE ENERGY
    # THIS IS NECESSARY BECAUSE ENERGY ALREADY IN MET IS NOT THE SAME AS THIS ENERGY (ASSUMED PZ WAS 0)
    lep_nu["E"] = np.sqrt(lep_nu.px**2 + lep_nu.py**2 + lep_nu.pz**2)

    print(f"Finished calculation of neutrino 4momenta after {time.time()-lep_time:.2f} seconds.")
    print()
    
    # RECONSTRUCTING W BOSON CANDIDATE
    w_lep_cand = lep_mu + lep_nu
    
    # CUTTING W CANDIDATE MASS TO PRODUCE BETTER RESULTS FOR TOP QUARK
    w_cut = (w_lep_cand.mass > 60) & (w_lep_cand.mass < 100)
    
    # TOP QUARK CANDIDATE RECONSTRUCTION
    top_lep_cand = w_lep_cand[w_cut] + lep_bjet[w_cut]

    print()
    print(f"Finished Leptonic Reconstruction after {time.time()-lep_time:.2f} seconds.")  
    print("=============================================================")
    
    return top_had_cand, w_had_cand, top_lep_cand, w_lep_cand


In [ ]:
def main(filename=None, events_keys=None, plot=False, pt_cut_vals=[0], btag_cut="Tight", reconstruct=False):
    """
    Main calling function for process_file and plot_func and reconstructor functions

    Example usage:
    events_big, top_had_cand, w_had_cand, top_lep_cand, w_lep_cand = main(events_keys=big_events, reconstruct=True, plot=True)
    
    ############################
    ########## INPUTS ##########
    ############################
    
    filename (str, default=None) - Full root file destination
    
    events_keys (uproot.model (Tree), default=None) - Root file keys; Can be entered to skip re-processing of root file
    
    plot (bool, default=False) - Calls plotting function if set to True
    
    cut_vals (int/float list, default=[0]) - Transverse momentum cut values
    
    btag_cut (str, default="none") - None, Loose, Medium, Tight cut using Jet_btagDeepB and Jet_btagDeepFlavB

    reconstruct (bool, default=False) - Calls reconstructor function to make top quark and w boson
                                        If plot is also true, makes invariant mass plots

    #############################
    ########## RETURNS ##########
    #############################
    
    events (uproot.model (Tree)) - Root file keys

    If reconstruct==True:
        top_had_cand (Awkward Array Record Type) - values of top quark from hadronic reconstruction
        w_had_cand (Awkward Array Record Type) - values of w boson from hadronic reconstruction
        top_lep_cand (Awkward Array Record Type) - values of top quark from leptonic reconstruction
        w_lep_cand (Awkward Array Record Type) - values of w boson from leptonic reconstruction
    """
    print()
    print("Main function called, please hold...")
    start_time = time.time()

    
    ########################################
    ########## GETTING EVENT TREE ##########
    ########################################
    if events_keys is None and filename is not None:
        print()
        print(f"Processing input file: {filename}")
        events = process_file(filename)
        print(f"Processing finished after {time.time() - start_time:.2f} seconds.")
    
    elif events_keys is not None and filename is None:
        print("Using input events tree, skipping processing function!")
        events = events_keys
    elif events_keys is None and filename is None:
        print("No input for 'events_keys' and 'filename'.")
        stop = r"""
        _          _            _            _      
       / /\       /\ \         /\ \         /\ \    
      / /  \      \_\ \       /  \ \       /  \ \   
     / / /\ \__   /\__ \     / /\ \ \     / /\ \ \  
    / / /\ \___\ / /_ \ \   / / /\ \ \   / / /\ \_\ 
    \ \ \ \/___// / /\ \ \ / / /  \ \_\ / / /_/ / / 
     \ \ \     / / /  \/_// / /   / / // / /__\/ /  
 _    \ \ \   / / /      / / /   / / // / /_____/   
/_/\__/ / /  / / /      / / /___/ / // / /          
\ \/___/ /  /_/ /      / / /____\/ // / /           
 \_____\/   \_\/       \/_________/ \/_/            
                                                    

        """
        print(stop)
        return
    ############################################################
    ########## PERFORMING RECONSTRUCTION AND PLOTTING ##########
    ############################################################
    if reconstruct == True and plot == True:
        print()
        print("Reconstructing and plotting...")
        print()
        
        read_time = time.time()
        muons, jets, met = events_to_list(events)
        print(f"It took {time.time() - read_time:.2f} seconds to access all this data.")
        
        top_had_cand, w_had_cand, top_lep_cand, w_lep_cand = reconstructor(muons=muons, jets=jets, met=met, btag_cut=btag_cut)

        #top_w = zip(top_had_cand, w_had_cand, top_lep_cand, w_lep_cand)
        print(f"Finished Reconstruction after {time.time()-start_time:.2f} seconds.")

        plot_func(reconstruct=True, btag_cut=btag_cut, top_had_cand=top_had_cand, w_had_cand=w_had_cand, top_lep_cand=top_lep_cand, w_lep_cand=w_lep_cand);

        print(f"Main finished running after {time.time() - start_time:.2f} seconds.")
        
        return events, top_had_cand, w_had_cand, top_lep_cand, w_lep_cand
        
    #########################################
    ########## ONLY RECONSTRUCTION ##########
    #########################################
    elif reconstruct == True and plot == False:
        read_time = time.time()
        muons, jets, met = events_to_list(events)
        print(f"It took {time.time() - read_time:.2f} seconds to access all this data.")
        
        top_had_cand, w_had_cand, top_lep_cand, w_lep_cand = reconstructor(muons=muons, jets=jets, met=met)
        print(f"Finished Reconstruction after {time.time()-start_time:.2f} seconds.")
        print()
        print("Skipping Plotting.")
        print()
        print(f"Main finished running after {time.time() - start_time:.2f} seconds.")
        
        return events, top_had_cand, w_had_cand, top_lep_cand, w_lep_cand

    ###################################
    ########## ONLY PLOTTING ##########
    ###################################
    elif reconstruct == False and plot == True:
        print("Skipping reconstruction.")
        plot_time = time.time()
        plot_func(events, pt_cut_vals, btag_cut);
        print()
        print(f"Finished plotting after {time.time() - plot_time:.2f} seconds.")
        print()
        print(f"Main finished running after {time.time() - start_time:.2f} seconds.")

        return events
        
    ######################################
    ########## ONLY EVENTS TREE ##########
    ######################################
    else:
        print()
        print(f"Main finished running after {time.time() - start_time:.2f} seconds.")
        print()
        return events

## Everything Else

In [ ]:
events = main(filename)

In [ ]:
jet_pt = events["Jet_pt"].array()
muon_pt = events["Muon_pt"].array()
print(f"Number of jets: {len(ak.flatten(jet_pt))}")
print(f"Number of muons: {len(ak.flatten(muon_pt))}")

In [ ]:
main(events_keys=big_events, pt_cut_vals=[20], btag_cut="Loose", plot=True);

In [ ]:
main(events_keys=big_events, pt_cut_vals=[20], btag_cut="Medium", plot=True)

In [ ]:
main(events_keys=big_events, pt_cut_vals=[20], btag_cut="Tight", plot=True)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)

plt.hist(ak.flatten(jet_btagB), bins=100, range=(0,1));
for val in btag_cut_dict:
    plt.axvline(btag_cut_dict[val][0], color=btag_cut_dict[val][-1], linestyle='--', label=val)
plt.title("Jet_btagDeepB")
plt.legend();

plt.subplot(1,2,2)

plt.hist(ak.flatten(jet_btag_flavB), bins=100, range=(0,1));
for val in btag_cut_dict:
    plt.axvline(btag_cut_dict[val][1], color=btag_cut_dict[val][-1], linestyle='--', label=val)
plt.title("Jet_btagDeepFlavB")
plt.legend();

# Invariant Mass of Top Quark and W Boson

In [ ]:
events = real_events

In [ ]:
pretty_print(events.keys(), fmt='40s', require=['Muon'], ignore='HLT')

In [ ]:
start_time = time.time()
# Muons -------------------------------------------------------
muon_pt = events['Muon_pt'].array()
muon_eta = events['Muon_eta'].array()
muon_phi = events['Muon_phi'].array()
muon_mass = events['Muon_mass'].array()

print(f"It took {time.time() - start_time:.2f} seconds to access Muons")
    
# Jets -------------------------------------------------------
start_time1 = time.time()

jet_pt = events['Jet_pt'].array()
jet_eta = events['Jet_eta'].array()
jet_phi = events['Jet_phi'].array()
jet_mass = events['Jet_mass'].array()
jets_btagDeepB = events["Jet_btagDeepB"].array()

print(f"It took {time.time() - start_time1:.2f} seconds to access Jets")

# MET ---------------------------------------------------------
start_time2 = time.time()

met_pt = events['PuppiMET_pt'].array()
met_eta = 0*events['PuppiMET_pt'].array()  # Fix this to be 0
met_phi = events['PuppiMET_phi'].array() 

print(f"It took {time.time() - start_time2:.2f} seconds to access MET")

######################################
print(f"It took {time.time() - start_time:.2f} seconds to access all this data")

In [ ]:
muons = ak.zip(
    {"pt": muon_pt, 
     "eta": muon_eta, 
     "phi": muon_phi, 
     "mass": muon_mass},
    with_name="Momentum4D",
)

jets = ak.zip(
    {"pt": jet_pt, 
     "eta": jet_eta, 
     "phi": jet_phi, 
     "mass": jet_mass,
     "btagDeepB": jets_btagDeepB,
     "btagDeepFlavB": jets_btagDeepFlavB},
    with_name="Momentum4D",
)

met = ak.zip(
    {"pt": met_pt, 
     "eta": met_eta, 
     "phi": met_phi, 
     "mass": np.zeros(len(met_phi))}, # We assume this is a neutrino with 0 mass
    with_name="Momentum4D",
)

In [ ]:
# Calculate all the different combinations
p4mu,p4j,p4met = ak.unzip(ak.cartesian([muons, jets, met]))

# Calculate a sum of the 4-momenta
p4tot = p4mu + p4j + p4met

In [ ]:
# Get the mass
x = p4tot.mass

print(x)

#ncand_cut = ak.num(x)==1
ncand_cut = ak.num(x)>0

# Plot it!
# Your code here
plt.figure()
plt.hist(ak.flatten(x[ncand_cut]), bins=40, range=(0,4000));
plt.hist(x[ncand_cut][:,0], bins=40, range=(0,4000));

In [ ]:
#########################################################################
##################### SELECTING DESIRED DATA (CUTS) #####################
#########################################################################

muon_mask = (muons.pt > 20) & (np.abs(muons.eta) < 2.1)
jet_mask = (jets.pt > 20) & (np.abs(jets.eta) < 2.4)

cut_muons = muons[muon_mask]
cut_jets = jets[jet_mask]

n_muons = ak.num(cut_muons)
n_jets = ak.num(cut_jets)
n_bjets = ak.sum(jets.btagDeepB > 0.8767, axis=1)

event_mask = (n_muons >= 1) & (n_jets >= 4) & (n_bjets == 2)

selected_jets = cut_jets[event_mask]
selected_muons = cut_muons[event_mask]

#################################################################
##################### TOP QUARK CALCULATION #####################
#################################################################

btag_cut = 0.8767
non_btag_cut = 0.3

trijets = ak.combinations(selected_jets, 3, fields=["j1","j2","j3"])
trijets["p4"] = trijets.j1 + trijets.j2 + trijets.j3 # Sum of 4-momenta

mask_jets_btag = ((trijets.j1.btagDeepB > btag_cut) & (trijets.j2.btagDeepB < non_btag_cut) & (trijets.j3.btagDeepB < non_btag_cut)) | \
                 ((trijets.j1.btagDeepB < non_btag_cut) & (trijets.j2.btagDeepB > btag_cut) & (trijets.j3.btagDeepB < non_btag_cut)) | \
                 ((trijets.j1.btagDeepB < non_btag_cut) & (trijets.j2.btagDeepB < non_btag_cut) & (trijets.j3.btagDeepB > btag_cut))

top_trijet = trijets.p4[mask_jets_btag][ak.argmax(trijets.p4.pt[mask_jets_btag], axis=1, keepdims=True)]
invmass_top = ak.flatten(top_trijet.mass)

###############################################################
##################### W BOSON CALCULATION #####################
###############################################################

event_mask = (n_muons >= 1) & (n_jets == 4)

selected_jets = cut_jets[event_mask]
selected_muons = cut_muons[event_mask]

dijets = ak.combinations(selected_jets, 2, fields=["j1","j2"])
dijets["p4"] = dijets.j1 + dijets.j2


non_mask_jets_btag = ((dijets.j1.btagDeepB < btag_cut) & (dijets.j2.btagDeepB < btag_cut))


dijets_nb = dijets[non_mask_jets_btag]
mass_dif = abs(dijets_nb.p4.mass - 80.4)
w_cand_had2 = dijets_nb.p4[ak.argmin(mass_dif, axis = 1, keepdims=True)]
invmass_w2 = ak.flatten(w_cand_had2.mass)

w_cand_had = dijets.p4[non_mask_jets_btag][ak.argmax(dijets.p4.pt[non_mask_jets_btag], axis=1, keepdims=True)]
invmass_w = ak.flatten(w_cand_had.mass)



In [ ]:
accepted_topq = 172.76 # GeV
accepted_wbos = 80.4 # GeV

plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("Top Quark Candidate Invariant Mass", fontsize=16)
plt.hist(invmass_top, bins=100, range=(0,400),label=f"btag cut: {btag_cut}")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value ({accepted_topq} GeV)")

plt.xticks(np.arange(0,275,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]", fontsize=14)
plt.ylabel("Events", fontsize=14)
plt.legend();

plt.subplot(1,2,2)
plt.title("W Boson Candidate Invariant Mass", fontsize=16)
plt.hist(invmass_w2, bins=100, range=(50,150))
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value ({accepted_wbos} GeV)")

#plt.xticks(np.arange(0,275,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]", fontsize=14)
plt.ylabel("Events", fontsize=14)
plt.legend();

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("Top Quark Candidate Invariant Mass", fontsize=16)
plt.hist(invmass_top, bins=100, range=(0,400),label=f"btag cut: {btag_cut}")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value ({accepted_topq} GeV)")

plt.xticks(np.arange(0,275,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]", fontsize=14)
plt.ylabel("Events", fontsize=14)
plt.legend();

plt.subplot(1,2,2)
plt.title("W Boson Candidate Invariant Mass", fontsize=16)
plt.hist(invmass_w, bins=100, range=(0,250))
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value ({accepted_wbos} GeV)")

plt.xticks(np.arange(0,275,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]", fontsize=14)
plt.ylabel("Events", fontsize=14)
plt.legend();

In [ ]:
plt.figure(figsize=(20,5))


plt.subplot(1,2,1)
plt.title("W Boson Candidate Invariant Mass", fontsize=16)
plt.hist(invmass_w, bins=100, range=(0,250), label="v1")
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value ({accepted_wbos} GeV)")

plt.xticks(np.arange(0,275,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]", fontsize=14)
plt.ylabel("Events", fontsize=14)
plt.legend();

plt.subplot(1,2,2)
plt.title("W Boson Candidate Invariant Mass", fontsize=16)
plt.hist(invmass_w2, bins=100, range=(0,250), label="v2")
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value ({accepted_wbos} GeV)")

plt.xticks(np.arange(0,275,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]", fontsize=14)
plt.ylabel("Events", fontsize=14)
plt.legend();

In [ ]:
btag_cut = 0.5
non_btag_cut = 0.2


mask1 = ((trijets.j1.btagDeepB > btag_cut) & (trijets.j2.btagDeepB < non_btag_cut) & (trijets.j3.btagDeepB < non_btag_cut))
mask2 = ((trijets.j1.btagDeepB < non_btag_cut) & (trijets.j2.btagDeepB > btag_cut) & (trijets.j3.btagDeepB < non_btag_cut))
mask3 = ((trijets.j1.btagDeepB < non_btag_cut) & (trijets.j2.btagDeepB < non_btag_cut) & (trijets.j3.btagDeepB > btag_cut))

mask = mask1 | mask2 | mask3

n = 2
for i in range(len(trijets[n].j1)):
    
    print('------------')
    print(trijets[n].j1[i])
    print(trijets[n].j2[i])
    print(trijets[n].j3[i])
    print()
    print(mask1[n])
    print(mask2[n])
    print(mask3[n])
    print(mask[n])

# Leptonic Reconstruction of Top Quark

In [ ]:
muon_mask = (muons.pt > 20) & (np.abs(muons.eta) < 2.1)
jet_mask = (jets.pt > 20) & (np.abs(jets.eta) < 2.4)

cut_muons = muons[muon_mask]
cut_jets = jets[jet_mask]
cut_met = met
btag_cut_dict = {
"None"  : [0.0000, 0.0000, "black"],
"Loose" : [0.1918, 0.0480, "purple"],
"Medium": [0.5847, 0.2489, "yellow"],
"Tight" : [0.8767, 0.6377, "red"]}
btag_val = btag_cut_dict["Tight"][0]

n_muons = ak.num(cut_muons)
n_jets = ak.num(cut_jets)
n_nonbjets = ak.sum(jets.btagDeepB < btag_val, axis=1)
n_bjets = ak.sum(jets.btagDeepB > btag_val, axis=1)


event_mask2 = (n_muons >= 1) & (n_jets >= 4) & (n_bjets == 2) & (met.pt > 20)


selected_bjets = cut_jets[event_mask2]
selected_muons = cut_muons[event_mask2]
selected_met = cut_met[event_mask2]

bjet = ak.zip(
    {"px": selected_bjets[:,0].px,
     "py": selected_bjets[:,0].py,
     "pz": selected_bjets[:,0].pz,
     "E":  selected_bjets[:,0].energy},
    with_name="Momentum4D"
)

mu = ak.zip(
    {"px": selected_muons[:,0].px,
     "py": selected_muons[:,0].py,
     "pz": selected_muons[:,0].pz,
     "E":  selected_muons[:,0].energy},
    with_name="Momentum4D"
)

nu = ak.zip(
    {"px": selected_met.px,
     "py": selected_met.py,
     "pz": np.zeros(len(selected_met.py)),
     "E":  np.zeros(len(selected_met.py))},
    with_name="Momentum4D"
)

In [ ]:
MW = 80.4 # GeV
mu_nu_p = mu.px*nu.px + mu.py*nu.py

Dtmp = MW**2 - mu.mass**2 + 2*(mu_nu_p)
Atmp = 4*(mu.energy**2 - mu.pz**2)
Btmp = -4 * Dtmp * mu.pz
Ctmp = 4 * mu.energy**2 * nu.pt**2 - Dtmp**2

disc = Btmp**2 - 4*Atmp*Ctmp

nu_pz1 = (-Btmp + np.sqrt(disc))/(2*Atmp)
nu_pz2 = (-Btmp - np.sqrt(disc))/(2*Atmp)

real_sol = disc >= 0

nu["pz"] = ak.where(real_sol,
    ak.where(abs(nu_pz1) < abs(nu_pz2), nu_pz1, nu_pz2),
                    -Btmp/(2*Atmp)
)

nu["E"] = np.sqrt(nu.px**2 + nu.py**2 + nu.pz**2)

In [ ]:
w_cand_lep = mu + nu

In [ ]:
w_cut = (w_cand_lep.mass > 60) & (w_cand_lep.mass < 90)
top_cand_lep = w_cand_lep[w_cut] + bjet[w_cut]

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("Top Quark Candidate Invariant Mass",fontsize=16)
plt.hist(top_cand_lep.mass, bins=100, range=(120,500),label=f"btag cut: {btag_val}")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value ({accepted_topq} GeV)")

plt.xticks(np.arange(125,200,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();

plt.subplot(1,2,2)
plt.title("W Boson Candidate Invariant Mass",fontsize=16)
plt.hist(w_cand_lep.mass, bins=200, range=(75,275));
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value ({accepted_wbos} GeV)")
plt.yscale('log')

#plt.xticks(np.arange(0,275,25),minor=True)
#plt.xlim(70,150)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("Top Quark Candidate Invariant Mass",fontsize=16)
plt.hist(top_cand_lep.mass, bins=100, range=(0,500),label=f"btag cut: {btag_val}")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value ({accepted_topq} GeV)")

plt.xticks(np.arange(125,200,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();

plt.subplot(1,2,2)
plt.title("Top Quark Candidate Invariant Mass",fontsize=16)
plt.hist(top_cand_lep.mass, bins=100, range=(121,500),label=f"btag cut: {btag_val}")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value ({accepted_topq} GeV)")

plt.xticks(np.arange(125,200,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();

In [ ]:
events_big, top_w_big = main(events_keys=big_events, reconstruct=True, plot=True)

# Testing Grounds

## Bellis' Memory Leak

In [ ]:
jet_kinematic_mask = (jets.pt > 20) & (np.abs(jets.eta) < 2.4)

jet_bjet_mask = (jets.btagDeepB > btag_val)
jet_not_bjet_mask = (jets.btagDeepB < 0.5)

jets_b =     jets[jet_bjet_mask & jet_kinematic_mask]
jets_not_b = jets[jet_not_bjet_mask & jet_kinematic_mask]

n_bjets = ak.num(jets_b)
n_not_bjets = ak.num(jets_not_b)

plt.figure(figsize=(20, 5))

plt.subplot(1,2,1)
plt.hist(n_bjets, bins=5, range=(-0.5,4.5))
plt.subplot(1,2,2)
plt.hist(n_not_bjets, bins=15, range=(-0.5,14.5))

In [ ]:
mask_numbers = (n_bjets >= 2) & (n_not_bjets >=2)

bjets_combos = ak.combinations(jets_b[mask_numbers], 2)
not_bjets_combos = ak.combinations(jets_not_b[mask_numbers], 2)

In [ ]:
not_bjets_combos[1]

In [ ]:
bjets_combos[1]

## Collin's Nuclear Bomb Site

### Hadronic Reconstruction

In [ ]:
muon_mask = (muons.pt > 20) & (np.abs(muons.eta) < 2.1)
jet_kinematic_mask = (jets.pt > 20) & (np.abs(jets.eta) < 2.4)

cut_muons = muons[muon_mask]
cut_jets = jets[jet_kinematic_mask]

btag_val = btag_cut_dict["Tight"][0]

n_muons = ak.num(cut_muons)
n_jets = ak.num(cut_jets)

event_mask = (n_muons >= 1) & (n_jets >= 4) & (met.pt > 30)


selected_jets = cut_jets[event_mask2]
selected_muons = cut_muons[event_mask2]
selected_met = met[event_mask2]


jet_bjet_mask = (selected_jets.btagDeepB > btag_val)
jet_not_bjet_mask = (selected_jets.btagDeepB < 0.5)

jets_b =     selected_jets[jet_bjet_mask]
jets_not_b = selected_jets[jet_not_bjet_mask]

n_bjets = ak.num(jets_b)
n_not_bjets = ak.num(jets_not_b)

mask_jets = (n_bjets >= 2) & (n_not_bjets >=2)

bjets_combos = ak.combinations(jets_b[mask_jets], 2, fields=["b1","b2"])
not_bjets_combos = ak.combinations(jets_not_b[mask_jets], 2, fields=["j1","j2"])

In [ ]:
combo = ak.cartesian({"bjets":bjets_combos, "jets":not_bjets_combos})

In [ ]:
w_cand = ak.flatten(combo.jets.j1 + combo.jets.j2)
w_mass_mask = (w_cand.mass > 60) & (w_cand.mass < 100)

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("W Boson Candidate Invariant Mass (no mass mask)",fontsize=16)
plt.hist(w_cand.mass, bins=100, range=(0,200),label=f"btag cut: {btag_val}")
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value ({accepted_wbos} GeV)")

plt.xticks(np.arange(125,200,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();



plt.subplot(1,2,2)
plt.title("W Boson Candidate Invariant Mass (with mass mask)",fontsize=16)
plt.hist(w_cand[w_mass_mask].mass, bins=100,label=f"btag cut: {btag_val}")
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value ({accepted_wbos} GeV)")

#plt.xticks(np.arange(125,200,25),minor=True)
#plt.yscale('log')
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();

In [ ]:
b1_combo = ak.flatten(combo.bjets.b1)
b2_combo = ak.flatten(combo.bjets.b2)

In [ ]:
top_had_1 = b1_combo[w_mass_mask] + w_cand[w_mass_mask]
top_had_2 = b2_combo[w_mass_mask] + w_cand[w_mass_mask]

In [ ]:
# top_had_1 = b1_combo + w_cand
# top_had_2 = b2_combo + w_cand

plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("Top Quark Candidate 1 Invariant Mass",fontsize=16)
counts1, bins1, patches1 = plt.hist(top_had_1.mass, bins=100, range=(0,500),label=f"btag cut: {btag_val}")


peak_bin_idx1 = np.argmax(counts1) ; peak_pos1 = (bins1[peak_bin_idx1] + bins1[peak_bin_idx1 + 1]) / 2

plt.axvline(peak_pos1,color="salmon",linestyle="--",label=f"Peak: {peak_pos1:.2f} GeV")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value: {accepted_topq} GeV")


plt.xticks(np.arange(125,200,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();



###############################################################################################################
###############################################################################################################
###############################################################################################################
###############################################################################################################



plt.subplot(1,2,2)
plt.title("Top Quark Candidate 2 Invariant Mass",fontsize=16)
counts2, bins2, patches2 = plt.hist(top_had_2.mass, bins=100, range=(0,500),label=f"btag cut: {btag_val}")


peak_bin_idx2 = np.argmax(counts2) ; peak_pos2 = (bins2[peak_bin_idx2] + bins2[peak_bin_idx2 + 1]) / 2

plt.axvline(peak_pos2,color="salmon",linestyle="--",label=f"Peak: {peak_pos2:.2f} GeV")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value: {accepted_topq} GeV")


plt.xticks(np.arange(125,200,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();

In [ ]:
if abs(accepted_topq - peak_pos1) < abs(accepted_topq - peak_pos2):
    lep_bjet_set = combo.bjets.b2
    print("Top Quark Candidate 1 is closer to accepted value.")
    print("Bjet 2 has been chosen for leptonic reconstruction.")
else:
    lep_bjet_set = combo.bjets.b1
    print("Top Quark Candidate 2 is closer to accepted value.")
    print("Bjet 1 has been chosen for leptonic reconstruction.")

### Leptonic Reconstruction

In [ ]:
# Assigning values to zip for easy access to data and indexing and math

###########
## MUONS ##
###########
lep_mu = ak.zip(
    {"px": selected_muons[mask_jets][:,0].px,
     "py": selected_muons[mask_jets][:,0].py,
     "pz": selected_muons[mask_jets][:,0].pz,
     "E":  selected_muons[mask_jets][:,0].energy},
    with_name="Momentum4D"
)

###############
## NEUTRINOS ##
###############
lep_nu = ak.zip(
    {"px": selected_met[mask_jets].px,
     "py": selected_met[mask_jets].py,
     "pz": np.zeros(len(selected_met[mask_jets].py)),
     "E":  np.zeros(len(selected_met[mask_jets].py))},
    with_name="Momentum4D"
)

###########
## BJETS ##
###########
lep_bjet = ak.zip(
    {"px": lep_bjet_set[:,0].px,
     "py": lep_bjet_set[:,0].py,
     "pz": lep_bjet_set[:,0].pz,
     "E":  lep_bjet_set[:,0].energy},
    with_name="Momentum4D"
)

In [ ]:
# CALCULATION OF NEUTRINO PZ AND ENERGY
MW = 80.4 # GeV
mu_dot_nu = lep_mu.px*lep_nu.px + lep_mu.py*lep_nu.py # dot product between mu <px,py> and nu <px,py>

# QUADRATIC EQUATION VARIABLES
Dtmp = MW**2 - lep_mu.mass**2 + 2*(mu_dot_nu)
Atmp = 4*(lep_mu.energy**2 - lep_mu.pz**2)
Btmp = -4 * Dtmp * lep_mu.pz
Ctmp = 4 * lep_mu.energy**2 * lep_nu.pt**2 - Dtmp**2

disc = Btmp**2 - 4*Atmp*Ctmp

nu_pz1 = (-Btmp + np.sqrt(disc))/(2*Atmp)
nu_pz2 = (-Btmp - np.sqrt(disc))/(2*Atmp)

# GRABBING ONLY REAL SOLUTIONS
real_sol = disc >= 0

# CONDITIONAL TO TAKE THE BEST SOLUTION FOR NU PZ
lep_nu["pz"] = ak.where(real_sol, # equal to where real solution is true
    ak.where(abs(nu_pz1) < abs(nu_pz2), nu_pz1, nu_pz2), # deciding which one is greater to choose the best solution
                    -Btmp/(2*Atmp) # Otherwise forcing real solution
)

# CALCULATIING THE ENERGY
# THIS IS NECESSARY BECAUSE ENERGY ALREADY IN MET IS NOT THE SAME AS THIS ENERGY (ASSUMED PZ WAS 0)
lep_nu["E"] = np.sqrt(lep_nu.px**2 + lep_nu.py**2 + lep_nu.pz**2)

In [ ]:
# RECONSTRUCTING W BOSON CANDIDATE
lep_w_cand = lep_mu + lep_nu

# CUTTING W CANDIDATE MASS TO PRODUCE BETTER RESULTS FOR TOP QUARK
w_cut = (lep_w_cand.mass > 60) & (lep_w_cand.mass < 100)

# TOP QUARK CANDIDATE RECONSTRUCTION
lep_top_cand = lep_w_cand[w_cut] + lep_bjet[w_cut]

In [ ]:
plt.figure(figsize=(20,5))

plt.subplot(1,2,1)
plt.title("Leptonic Top Quark Candidate Invariant Mass",fontsize=16)
counts1, bins1, patches1 = plt.hist(lep_top_cand.mass, bins=100, range=(0,500),label=f"btag cut: {btag_val}")


peak_bin_idx1 = np.argmax(counts1) ; peak_pos1 = (bins1[peak_bin_idx1] + bins1[peak_bin_idx1 + 1]) / 2

plt.axvline(peak_pos1,color="salmon",linestyle="--",label=f"Peak: {peak_pos1:.2f} GeV")
plt.axvline(accepted_topq,color="red",linestyle="--",label=f"Known Value: {accepted_topq} GeV")


plt.xticks(np.arange(125,200,25),minor=True)
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();



###############################################################################################################
###############################################################################################################
###############################################################################################################
###############################################################################################################



plt.subplot(1,2,2)
plt.title("Leptonic W Boson Candidate Invariant Mass",fontsize=16)
counts2, bins2, patches2 = plt.hist(lep_w_cand.mass, bins=100, range=(75,275),label=f"btag cut: {btag_val}")


peak_bin_idx2 = np.argmax(counts2) ; peak_pos2 = (bins2[peak_bin_idx2] + bins2[peak_bin_idx2 + 1]) / 2

plt.axvline(peak_pos2,color="salmon",linestyle="--",label=f"Peak: {peak_pos2:.2f} GeV")
plt.axvline(accepted_wbos,color="red",linestyle="--",label=f"Known Value: {accepted_wbos} GeV")


#plt.xticks(np.arange(125,200,25),minor=True)
plt.yscale('log')
plt.xlabel("Invariant Mass [GeV/$c^2$]",fontsize=14)
plt.ylabel("Events",fontsize=14)
plt.legend();

In [ ]:
# def bongo():
#     num = np.random.randint(5)
#     if num == 0:
#         return 'boom'
#     elif num == 1:
#         return 'dum'
#     elif num == 2:
#         return 'bong'
#     elif num == 3:
#         return 'baddum'
#     else:
#         return 'bap'

In [ ]:
# i = 0
# while i < 1:
#     print(bongo())
#     i += 0.01